In [222]:
# Step 1: Import essentials
import pandas as pd   
import numpy as np   
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.metrics.pairwise import cosine_similarity     
import warnings


In [223]:
warnings.filterwarnings("ignore")  # to ignore warnings

In [224]:
# Load the dataset
data = pd.read_csv("investments_VC.csv", encoding="latin1", low_memory=False)
data.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [225]:
data.columns 

Index(['permalink', 'name', 'homepage_url', 'category_list', ' market ',
       ' funding_total_usd ', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'seed', 'venture', 'equity_crowdfunding',
       'undisclosed', 'convertible_note', 'debt_financing', 'angel', 'grant',
       'private_equity', 'post_ipo_equity', 'post_ipo_debt',
       'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H'],
      dtype='object')

In [226]:
data.shape

(54294, 39)

In [227]:
data.isnull().sum() #to check for null values

permalink                4856
name                     4857
homepage_url             8305
category_list            8817
 market                  8824
 funding_total_usd       4856
status                   6170
country_code            10129
state_code              24133
region                  10129
city                    10972
funding_rounds           4856
founded_at              15740
founded_month           15812
founded_quarter         15812
founded_year            15812
first_funding_at         4856
last_funding_at          4856
seed                     4856
venture                  4856
equity_crowdfunding      4856
undisclosed              4856
convertible_note         4856
debt_financing           4856
angel                    4856
grant                    4856
private_equity           4856
post_ipo_equity          4856
post_ipo_debt            4856
secondary_market         4856
product_crowdfunding     4856
round_A                  4856
round_B                  4856
round_C   

In [228]:
data.duplicated().sum() #to check for duplicate values

4855

In [229]:
data[data.duplicated()]  #to view the duplicate values

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
49439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
data = data.drop_duplicates()


In [231]:
data.shape

(49439, 39)

In [232]:
# Remove extra spaces from column names
data.columns = data.columns.str.strip()

# Check again
print(data.columns)


Index(['permalink', 'name', 'homepage_url', 'category_list', 'market',
       'funding_total_usd', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'seed', 'venture', 'equity_crowdfunding',
       'undisclosed', 'convertible_note', 'debt_financing', 'angel', 'grant',
       'private_equity', 'post_ipo_equity', 'post_ipo_debt',
       'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H'],
      dtype='object')


In [233]:
# Drop rows where name, category_list, or market are missing
data = data.dropna(subset=['name', 'category_list', 'market'])
data.shape 

(45469, 39)

In [234]:
# Convert funding_total_usd to numeric (remove non-numeric stuff)
data['funding_total_usd'] = (
    data['funding_total_usd']
    .replace('[\$,]', '', regex=True)   # remove $ and commas
    .replace('?', None)                 # handle weird values
)

# Convert to numeric type (floats/ints), errors='coerce' will turn invalid to NaN
data['funding_total_usd'] = pd.to_numeric(data['funding_total_usd'], errors='coerce')

# Fill missing values with 0 (or median/mean, depending on use case)
data['funding_total_usd'] = data['funding_total_usd'].fillna(0)

# Check results
print(data['funding_total_usd'].head())


0    1750000.0
1    4000000.0
2      40000.0
3    1500000.0
4      60000.0
Name: funding_total_usd, dtype: float64


In [235]:
# Remove leading/trailing "|" and extra spaces
data['category_list'] = data['category_list'].str.strip('|').str.strip()

# Then split and take the first category
data['main_category'] = data['category_list'].str.split('|').str[0]

# Check again
print(data[['category_list', 'main_category']].head(10))



                                       category_list  main_category
0           Entertainment|Politics|Social Media|News  Entertainment
1                                              Games          Games
2                               Publishing|Education     Publishing
3  Electronics|Guides|Coffee|Restaurants|Music|iP...    Electronics
4                        Tourism|Entertainment|Games        Tourism
5                                           Software       Software
6                                        Advertising    Advertising
7                                        Curated Web    Curated Web
8                                           Software       Software
9                                              Games          Games


In [236]:
data = data[['main_category', 'market', 'funding_total_usd',
            'funding_rounds', 'status', 'country_code', 'founded_year']]


In [237]:
data.shape

(45469, 7)

In [238]:
data.columns = data.columns.str.strip()


In [239]:
print(data.columns.tolist())


['main_category', 'market', 'funding_total_usd', 'funding_rounds', 'status', 'country_code', 'founded_year']


In [240]:
# Numerical
data['funding_rounds'] = data['funding_rounds'].fillna(data['funding_rounds'].median())
data['founded_year'] = data['founded_year'].fillna(data['founded_year'].median())

# Categorical
data['market'] = data['market'].fillna('Unknown')
data['status'] = data['status'].fillna('Unknown')
data['country_code'] = data['country_code'].fillna('Unknown')


In [241]:
data.head()

,main_category,market,funding_total_usd,funding_rounds,status,country_code,founded_year
0,Entertainment,News,1750000.0,1.0,acquired,USA,2012.0
1,Games,Games,4000000.0,2.0,operating,USA,2009.0
2,Publishing,Publishing,40000.0,1.0,operating,EST,2012.0
3,Electronics,Electronics,1500000.0,1.0,operating,GBR,2011.0
4,Tourism,Tourism,60000.0,2.0,operating,USA,2014.0


In [242]:
# Create a 'profile' column using original categorical columns BEFORE encoding
data['profile'] = (
    data['main_category'].astype(str) + " " +
    data['market'].astype(str) + " " +
    data['status'].astype(str)
)


In [243]:
data.head 

<bound method NDFrame.head of              main_category                 market  funding_total_usd  \
0            Entertainment                  News           1750000.0   
1                    Games                 Games           4000000.0   
2               Publishing            Publishing             40000.0   
3              Electronics           Electronics           1500000.0   
4                  Tourism               Tourism             60000.0   
...                    ...                    ...                ...   
49433            Analytics             Education            320000.0   
49434  Enterprise Software   Enterprise Software           1587301.0   
49435      Web Development       Web Development             97398.0   
49436                Games                 Games           9300000.0   
49437  Enterprise Software   Enterprise Software          45000000.0   

       funding_rounds     status country_code  founded_year  \
0                 1.0   acquired          

In [244]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['profile'])


In [245]:
from sklearn.metrics.pairwise import cosine_similarity
def recommend_startups(user_input, top_n=5):
    user_profile = " ".join(user_input)
    user_vec = vectorizer.transform([user_profile])
    similarity = cosine_similarity(user_vec, tfidf_matrix).flatten()
    top_indices = similarity.argsort()[-top_n:][::-1]
    
    # Return only columns that exist
    return data.iloc[top_indices][['main_category', 'market', 'funding_total_usd', 'status', 'country_code', 'founded_year']]




In [248]:
# New example user input
user_input = ['electronics', 'software', 'USA']

# Get top 5 recommendations
recommendations = recommend_startups(user_input, top_n=5)
print(recommendations)


      main_category         market  funding_total_usd     status country_code  \
37173      Software   Electronics         113200000.0  operating          ESP   
14050      Software   Electronics             17071.0  operating      Unknown   
17980      Software   Electronics            500000.0  operating          USA   
11004      Software   Electronics          12000000.0  operating          USA   
19962   Electronics      Software          30000000.0  operating      Unknown   

       founded_year  
37173        2009.0  
14050        2014.0  
17980        2012.0  
11004        2009.0  
19962        1993.0  
